In [ ]:
%matplotlib inline
#libraries for visualization and matrix/functions
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

#pandas is like dplyr/tidyr, vertica_python is needed to connect with vertica
from config import v_user, v_password
import pandas as pd
import vertica_python
import matplotlib.pyplot as plt
conn_info = {'host': 'devlx206',
             'port': 5433,
             'user': v_user,
             'password': v_password,
             'database': 'advana',
             # 10 minutes timeout on queries
             'read_timeout': 600,
             # default throw error on invalid UTF-8 results
             'unicode_error': 'strict',
             # SSL is disabled by default
             'ssl': True}
             
# simple connection, with manual close
connection = vertica_python.connect(**conn_info)
cur = connection.cursor()

In [6]:
#Pull Down data from current model using SQL
query = '''
select
    a.name_polnum as polnum,
    a.name_firstname as fn,
    a.name_lastname as ln,
    a.name_idnumber as ssn,
    h.iss_dt,
    h.rptd_dt,
    d.sum_entrydate,
    d.sum_dateaction,
    d.sum_agentcode,
    d.sum_agency_code,
    d.sum_status as status,
    d.sum_app_type as app_type
from
    winrisk.Name a
LEFT JOIN    winrisk.summary d on a.name_polnum = d.sum_polnum

LEFT JOIN    (select
    policy_no,
    min(case when value2='MAIL' then date_occured else cast('9999-12-31' as date) end) as iss_dt,
    min(case when value2='RPTD' then date_occured else cast('9999-12-31' as date) end) as rptd_dt
from
    winrisk.Policy_Events 
group by
    policy_no) h on a.name_polnum = h.policy_no
where
    a.name_role = 1;'''
PolData = pd.read_sql(query, connection)
PolData.to_csv("TrainingData.csv")